In [150]:
%run "0. INSTALLS & GLOBALS.ipynb"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [151]:
MASTER_DF = pd.read_csv("outputs/Nobel_Laureates_Clean.csv")
MASTER_DF[:10]

,YEAR,LAUREATE,NAME_ORDER,URL,FIELD
0,1901,Jacobus Henricus van 't Hoff,1,/wiki/Jacobus_Henricus_van_%27t_Hoff,Chemistry
1,1901,Sully Prudhomme,1,/wiki/Sully_Prudhomme,Literature
2,1901,Emil von Behring,1,/wiki/Emil_von_Behring,Medicine
3,1901,Henry Dunant,1,/wiki/Henry_Dunant,Peace
4,1901,Frédéric Passy,2,/wiki/Fr%C3%A9d%C3%A9ric_Passy,Peace
5,1901,Wilhelm Röntgen,1,/wiki/Wilhelm_R%C3%B6ntgen,Physics
6,1902,Emil Fischer,1,/wiki/Emil_Fischer,Chemistry
7,1902,Theodor Mommsen,1,/wiki/Theodor_Mommsen,Literature
8,1902,Ronald Ross,1,/wiki/Ronald_Ross,Medicine
9,1902,Élie Ducommun,1,/wiki/%C3%89lie_Ducommun,Peace


# Get Biography
DOB, DOD (Date of Death) (I don't know what else to call it.), Place of Birth, Place of Death, Date of Nobel Award

In [ ]:
import re

biography_ioi = ['born', 'died', 'nationality', 'alma mater', 'notable awards', 'awards']
special_characters = """!@#$%^&*()-+?_=,<>/"""

def extract_text(item, xpath_expr):
    """Helper function to get text from an XPath query and clean it."""
    return [text.strip() for text in item.xpath(xpath_expr) if text.strip() and text not in ["\n", " "]]

def process_born_died_date(row):
    ## get born/died dates
    td_text = extract_text(row, ".//*[contains(@style, 'none')]//text()")
    if td_text:
        value = ''.join(td_text)
        return value[1:-1].strip()
    else:
        td_text = extract_text(row, ".//td//text()[not(ancestor::style or ancestor::sup or ancestor::a)]")
        for words in td_text:
            if words[:3].isdigit():
                return words.split()[0]
    return False #something went wrong or this person is alive

def process_born_died_loc(row):
    td_text = extract_text(row, ".//text()[not(ancestor::style or ancestor::sup)]")
    temp = td_text[-2] if td_text and td_text[-1] in special_characters else td_text[-1]
    if len(temp.split(",")) >= 2: # "Paris, France"
        value = temp.split(",")[-1].strip()
    else:
        value = temp.replace(",", "").strip()
    return value

def process_awards(row):
    td_text = extract_text(row, ".//text()[not(ancestor::style or ancestor::sup)]")
    value = int((len(td_text) / 2) - 1) #awards will always be award-year pair. -1 because we are not including nobel prize
    return value

def process_almamater(row):
    td_text = extract_text(row, ".//td//text()[not(ancestor::style or ancestor::sup)]")
    return td_text
                

"""
def process_born_died:

label_processors = {
    "born": process_born_died,
    "died": process_born_died,
    "nationality": process_generic,
    "other awards": process_awards,
}"
"""

def get_biography(page):
    biography = {}
    biography_table = page.xpath("//*[contains(@class, 'vcard')]//tr")

    #if sibling element have [word], scrape content
    #im still mad this doesn't work. my work would've been so easy
    #born = biography_table[0].xpath(".//tr/th[contains(., 'born')]/following-sibling::*")
    
    for row in biography_table:
        th_text = extract_text(row, ".//th//text()")
        label = ''.join(th_text).lower().strip()
        #print(label)
        if label in ["born", "died"]:
            ## ASSIGN BORN_DIED DATE
            td_text = process_born_died_date(row)
            biography[label] = [td_text]

            ## ASSIGN BORN_DIED LOCATION
            label = "place of birth" if label == "born" else "place of death"
            td_text = process_born_died_loc(row)
            biography[label] = [td_text]
        ## ASSIGN NUMBER OF AWARDS
        elif "award" in label:
            label = "other awards"
            td_text = process_awards(row)
            if int(td_text) > 0:
                biography[label] = [td_text]
        ## ASSIGN ALMA MATERS
        elif label.startswith("alma"):
            td_text = process_almamater(row)

        if biography.get("died"):
            biography["age"] = biography["born"] 
        else:
            biography["age"] = biography["born"] 
        
    if biography:
        biography["type"] = "Individual"
    else:
        biography["type"] = "Institute"

    return biography

# Driver Code

In [290]:
temp = []
index = 0

for row in MASTER_DF[1000:].itertuples():
    laureate_url = row[4]
    url = WIKIPEDIA_URL + laureate_url
    #debug
    print(f"{index}. Scraping: {url}", flush=True)
    index+=1

    bio_data = {}
    if laureate_url != "No Record":
        page = GET_ETREE(DRIVER, url, parser="html")
        bio_data = get_biography(page) #only works for individuals; institutes will be added later
        print(bio_data)

    temp.append(bio_data)

0. Scraping: https://en.wikipedia.org/wiki/Anne_L%27Huillier
{'born': ['1958-08-16'], 'place of birth': ['France'], 'other awards': [3], 'type': 'Individual'}
1. Scraping: https://en.wikipedia.org/wiki/David_Baker_(biochemist)
{'born': ['1962-10-06'], 'place of birth': ['U.S.'], 'other awards': [6], 'type': 'Individual'}
2. Scraping: https://en.wikipedia.org/wiki/Demis_Hassabis
{'born': ['1976-07-27'], 'place of birth': ['England'], 'other awards': [6], 'type': 'Individual'}
3. Scraping: https://en.wikipedia.org/wiki/John_M._Jumper
{'born': ['1985'], 'place of birth': ['U.S.'], 'other awards': [4], 'type': 'Individual'}
4. Scraping: https://en.wikipedia.org/wiki/Daron_Acemoglu
{'born': ['1967-09-03'], 'place of birth': ['Turkey'], 'other awards': [4], 'type': 'Individual'}
5. Scraping: https://en.wikipedia.org/wiki/Simon_Johnson_(economist)
{'born': ['1963-01-16'], 'place of birth': ['United Kingdom'], 'type': 'Individual'}
6. Scraping: https://en.wikipedia.org/wiki/James_A._Robinson
{

In [ ]:
df = pd.DataFrame(temp)
biography_df = MASTER_DF.join(df)
biography_df[980:-1]

,YEAR,LAUREATE,NAME_ORDER,URL,FIELD,place of birth,born,place of death,age at death,died,other awards,nationality,alma mater
980,2022,Douglas Diamond,2,/wiki/Douglas_Diamond,Economy,U.S.,1953-10-25,NaN,NaN,NaN,0.0,NaN,NaN
981,2022,Philip H. Dybvig,3,/wiki/Philip_H._Dybvig,Economy,U.S.,1955-05-22,NaN,NaN,NaN,0.0,NaN,NaN
982,2022,Annie Ernaux,1,/wiki/Annie_Ernaux,Literature,France,1940-09-01,NaN,NaN,NaN,0.0,NaN,NaN
983,2022,Svante Pääbo,1,/wiki/Svante_P%C3%A4%C3%A4bo,Medicine,Sweden,1955-04-20,NaN,NaN,NaN,18.0,NaN,NaN
984,2022,Ales Bialiatski,1,/wiki/Ales_Bialiatski,Peace,Russia),1962-09-25,NaN,NaN,NaN,3.0,NaN,NaN
985,2022,Memorial,2,/wiki/Memorial_(society),Peace,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986,2022,Centre for Civil Liberties,3,/wiki/Center_for_Civil_Liberties_(human_rights...,Peace,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
987,2022,Alain Aspect,1,/wiki/Alain_Aspect,Physics,Lot-et-Garonne France,1947-06-15,NaN,NaN,NaN,4.0,NaN,NaN
988,2022,John Clauser,2,/wiki/John_Clauser,Physics,U.S.,1942-12-01,NaN,NaN,NaN,1.0,NaN,NaN
989,2022,Anton Zeilinger,3,/wiki/Anton_Zeilinger,Physics,Austria,1945-05-20,NaN,NaN,NaN,10.0,NaN,NaN


In [ ]:
#biography_df.to_csv("outputs/Nobel_Laureates_Biography.csv")

In [ ]:
biography_df["born"] = pd.to_datetime(df["born"]).dt.date
biography_df["died"] = pd.to_datetime(df["died"]).dt.date

biography_df["other awards"] = biography_df["other awards"].astype("int")

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
biography_df

,YEAR,LAUREATE,NAME_ORDER,URL,FIELD,place of birth,born,place of death,age at death,died,other awards,nationality,alma mater
0,1901,Jacobus Henricus van 't Hoff,1,/wiki/Jacobus_Henricus_van_%27t_Hoff,Chemistry,Netherlands,1852-08-30,Berlin German Empire,58,1911-03-01,4.0,NaN,NaN
1,1901,Sully Prudhomme,1,/wiki/Sully_Prudhomme,Literature,France,1839-03-16,France,68,1907-09-06,0.0,French,NaN
2,1901,Emil von Behring,1,/wiki/Emil_von_Behring,Medicine,Poland,1854-03-15,German Empire,63,1917-03-31,1.0,German,NaN
3,1901,Henry Dunant,1,/wiki/Henry_Dunant,Peace,Switzerland,1828-05-08,Heiden Switzerland,82,1910-10-30,0.0,Swiss,NaN
4,1901,Frédéric Passy,2,/wiki/Fr%C3%A9d%C3%A9ric_Passy,Peace,France,1822-05-20,France,90,1912-06-12,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,2024,Victor Ambros,1,/wiki/Victor_Ambros,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1009,2024,Gary Ruvkun,2,/wiki/Gary_Ruvkun,Medicine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,2024,Nihon Hidankyo,1,/wiki/Nihon_Hidankyo,Peace,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,2024,John Hopfield,1,/wiki/John_Hopfield,Physics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
